In [ ]:
#| default_exp examples

# examples

> API details.

In [ ]:
#|hide
from nbdev.showdoc import *


In [ ]:
#|export
import logging
logging.basicConfig(
    format='"%(asctime)s",%(name)s,%(funcName)s,%(levelname)s,"%(message)s"',
    level=logging.INFO)


In [ ]:
#|export
def example_01() -> None:
    '''
    Example 01
    Print debugging to the screen using print()
    '''
    print('This debug message uses print.')



In [ ]:
example_01()

This debug message uses print.


The next example uses logging instead of print(). 
Note that this is almost as simple as print().  In the following examples, we'll see why logging is better than print!

In [ ]:
#|export

def example_02() -> None:
    '''
    Example 02
    Log to the screen with the root logger.

    '''
    
    rootlogger = logging.getLogger()
    old_level = rootlogger.getEffectiveLevel()
    rootlogger.setLevel(level=logging.DEBUG)
    logging.debug('This debug message uses the root logger.')
    logging.getLogger().setLevel(old_level)





In [ ]:
example_02()

"2022-08-16 16:03:50,512",root,DEBUG,"This debug message uses the root logger."


We can get the root logger from anywhere in our application with logging.getLogger()
We already configured the root logger in example_02, but we can modify its behavior here.
This illustrates two handy things about logging:

- logging provides access to the loggers from anywhere, so the developer does not have to search the code.
- We can change the level of messages to suppress debug messages.

In [ ]:
#|export
def example_03() -> None:
    '''
    Example 03
    Add formatting to the root logger a custom format string.
    '''
    
    rootlogger = logging.getLogger()
    my_format = logging.Formatter('"%(asctime)s",%(name)s,%(levelname)s,"%(message)s"')
    rootlogger.handlers[0].setFormatter(my_format)
    
    logging.info('This uses the root logger with a custom format, adding context information to each message.')
    


In [ ]:
example_03()

"2022-08-16 16:03:50,554",root,INFO,"This uses the root logger with a custom format, adding context information to each message."


In [ ]:
#|export

def example_04():
    '''
    Example 04
    Use a named logger that has unique properties from the root logger.
    '''

    # Let's create a custom logger, my_logger.
    my_logger = logging.getLogger(__name__)
    my_logger.setLevel(logging.DEBUG)
    logging.info('This INFO message sent to the root logger will display.')
    logging.debug('We will not see this because DEBUG is lower than the root level INFO.')

    my_logger = logging.getLogger(__name__)
    my_logger.debug('The __main__ logger is set to a level of DEBUG, so this message displays.')
    my_logger.info('Note the __main__ logger uses the format from root.')



In [ ]:
example_04()

"2022-08-16 16:03:50,590",root,INFO,"This INFO message sent to the root logger will display."
"2022-08-16 16:03:50,591",__main__,DEBUG,"The __main__ logger is set to a level of DEBUG, so this message displays."
"2022-08-16 16:03:50,591",__main__,INFO,"Note the __main__ logger uses the format from root."


In [ ]:
#|export

def example_05():
    '''
    Example 05
    Log to a file with print()
    '''
    import sys

    with open(file='example_print.log', mode='a') as file:
        print('This is an example of writing to a logfile with print.', file=file)

In [ ]:
example_05()


In [ ]:
#|export

def example_06_get_logger(level=logging.INFO):
    '''
    Example 06
    Customize our logger with a file handler and a formatter.
    '''
    logger =  logging.getLogger(__name__)
    logger.setLevel(level)
    return logger
    
def example_06_configure_handler(filename=None, format=None, level=logging.INFO) -> logging.Handler:
    '''
    We create and customize a file logger with a logging.Handler that
    send log messages to a file.
     - add a custom formatter
    '''
    if filename is not None:
        handler = logging.FileHandler(filename=filename, mode='a')
        handler.setFormatter(fmt=logging.Formatter(format))
    else:
        handler = None
    return handler

def example_06():
    FORMAT = '"%(asctime)s",%(module)s,%(name)s,%(levelname)s,"%(message)s"'
    FILENAME='example_06.log'

    logger = example_06_get_logger(level=logging.INFO) # the screen handler won't show debug messages

    logger.addHandler(
        example_06_configure_handler(format=FORMAT, 
                                     filename=FILENAME, 
                                     level=logging.DEBUG)) # the file handler WILL show debug messages.

    logger.debug(f"Debug messages go to the file {FILENAME}")

In [ ]:
example_06()

In [ ]:
!cat example_06.log

In [ ]:
#|export
'''
Example 07
A logger that sends debug messages to the screen and info messages to a file
'''

FORMAT = '"%(asctime)s",%(module)s,%(funcName)s, %(name)s,%(levelname)s,"%(message)s"'
FILENAME='example_07.log'
CUSTOM_LEVEL = logging.DEBUG

formatter = logging.Formatter(fmt=FORMAT)

def example_07_get_logger() -> logging.getLogger:
    logger =  logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    return logger

def example_07_configure_file_handler(filename=FILENAME) -> logging.Handler:

    file_handler = logging.FileHandler(filename=FILENAME, mode='a')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    return file_handler

def example_07_configure_screen_handler() -> logging.Handler:
    import sys

    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setLevel(CUSTOM_LEVEL)
    screen_handler.setFormatter(formatter)

    return screen_handler

def example_07():
    logger = example_07_get_logger()
    logger.addHandler(example_07_configure_screen_handler())
    logger.addHandler(example_07_configure_file_handler())

    logger.debug('debug')
    logger.info("Info messages go to screen and file")

In [ ]:
example_07()

"2022-08-16 16:03:50,834",3089691349,example_07, __main__,DEBUG,"debug"


"2022-08-16 16:03:50,834",__main__,DEBUG,"debug"


"2022-08-16 16:03:50,834",3089691349,example_07, __main__,INFO,"Info messages go to screen and file"


"2022-08-16 16:03:50,834",__main__,INFO,"Info messages go to screen and file"
